In [1]:
import numpy as np

In [79]:
class MLP:
    def __init__(self,InputSize : int, hiddenLayers: list, OutputSize:int):
        self.weights = {}

        self.weights["W1"] = np.zeros((InputSize,hiddenLayers[0]))
        self.weights["B1"] = np.zeros(hiddenLayers[0])

        wn = 2
        for i in range(1,len(hiddenLayers)):
            self.weights["W"+str(wn)] = np.zeros((hiddenLayers[i-1],hiddenLayers[i]))
            self.weights["B"+str(wn)] = np.zeros(hiddenLayers[i])

            wn += 1

        self.weights["W"+str(wn)] = np.zeros((hiddenLayers[-1],OutputSize))
        self.weights["B"+str(wn)] = np.zeros(OutputSize)

    def activation(self,Z):
        return 1/(1+np.exp(-Z))

    def forward(self,X):

        self.outputs = []
        A = X
        for i in range(1,len(self.weights)//2 + 1):
            Z = A@self.weights["W"+str(i)] + self.weights["B"+str(i)]
            A = self.activation(Z)
            self.outputs.append(A)

        return A

    def backwardProp(self,X,y):
        ypred = self.forward(X)
        delta4 = ypred-y
        gradw4 = (self.outputs[2].T)@delta4

        
        
        delta3 = (delta4@self.weights['W4'].T)*(self.outputs[2]*(1-self.outputs[2]))
        gradw3 = (self.outputs[1].T)@delta3

        # print(self.weights["W3"].shape,delta3.shape)
        delta2 = (delta3@self.weights['W3'].T)*(self.outputs[1]*(1-self.outputs[1]))
        gradw2 = (self.outputs[0].T)@delta2

        delta1 = (delta2@self.weights['W2'].T)*(self.outputs[0]*(1-self.outputs[0]))
        gradw1 = (X.T)@delta1

        print(delta1.shape,delta2.shape,delta3.shape,delta4.shape)
        print(gradw1.shape,gradw2.shape,gradw3.shape,gradw4.shape)
        
        # self.weights["W"+str(wn)] = 
        

In [80]:
clf = MLP(3,[4,3,2],1)

In [81]:
X = np.array([[1,2,3],[4,5,6]])
clf.forward(X).shape

(2, 1)

In [82]:
clf.outputs

[array([[0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5]]),
 array([[0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5]]),
 array([[0.5, 0.5],
        [0.5, 0.5]]),
 array([[0.5],
        [0.5]])]

In [83]:
clf.backwardProp(X,np.array([0,1]).reshape((2,1)))

(2, 4) (2, 3) (2, 2) (2, 1)
(3, 4) (4, 3) (3, 2) (2, 1)


In [85]:
np.array([0,1]).reshape((2,1)).shape

(2, 1)